## util functions

In [13]:
import random
import math

# returns a matrix of random connection costs between cities
# costs between 1 and maxCost
def randomConnections(cities, maxCost):
    connections = []
    for i in range(cities):
        connections.append([0]*cities)
    for i in range(cities):
        for j in range(i+1,cities):
            connections[i][j] = random.randint(1, maxCost)
            connections[j][i] = connections[i][j]
    return connections

# changes cities visit order
def tweak(state, i, j):
    state[i], state[j] = state[j], state[i]

# returns random starting cities order
def randomState(cities):
    i=0
    state = list(range(cities))
    while(i<200):
        i+=1
        tweak(state, random.randint(0,cities-1), random.randint(0,cities-1))
    return state

# evaluates total travel cost
def evalState(state, connections):
    size = len(state)
    cost = 0
    for i in range(size-1):
        cost+=connections[state[i]][state[i+1]]
    cost+=connections[state[size-1]][state[0]]
    return cost

## simulated annealing

In [14]:
# returns a tweaked state from a starting state
def nextRandomState(state):
    size = len(state)
    i = random.randint(0,size-1)
    j = random.randint(0,size-1)
    while i==j: j = random.randint(0,size-1)
    next_state = state.copy()
    tweak(next_state, i, j)
    return next_state
    
# simulated annealing
def simulatedAnnealing(initialState, iterations, startingT, connections, alfa):
    bestState = initialState
    bestEval = evalState(bestState, connections)
    currentState = bestState
    currentEval = bestEval
    t = startingT
    while t>0.1:
        for _ in range(iterations):
            nextState = nextRandomState(currentState)
            nextEval = evalState(nextState, connections)
            if nextEval<currentEval:
                currentState = nextState
                currentEval = nextEval
                if nextEval<bestEval:
                    bestState = nextState
                    bestEval = nextEval
            else:
                dE = nextEval - currentEval
                p = math.exp(-dE/t)
                if random.random() < p:
                    currentState = nextState
                    currentEval = nextEval
        t*=alfa
    return bestState, bestEval

## tabu search

In [15]:
# initializes tabu tenure
def tabuTenureStart(dim):
    tabuTenure = dict()
    for i in range(dim-1):
        for j in range(i+1, dim):
            tabuTenure[(i,j)] = 0    
    return tabuTenure

# true if la mossa è legittima
def tabuCheck(tabuTenure, move):
    return tabuTenure[move]==0

# decrements all tabu values for tabu moves
def tabuDecrement(tabuTenure):
    for key in tabuTenure.keys():
        if tabuTenure[key] != 0:
            tabuTenure[key]-=1

# applies a move to a state (tweaks 2 cities)
def applyMove(state, move):
    state = state.copy()
    state[move[0]], state[move[1]] = state[move[1]], state[move[0]]
    return state

# tabu search
def tabuSearch(initialState, connections, tabuBan, iterations):
    currentState = initialState
    currentEval = evalState(currentState, connections)

    bestState = currentState.copy()
    bestEval = currentEval
    tabuTenure = tabuTenureStart(len(currentState))
    for _ in range(iterations):
        allMoves = list(filter(lambda move: tabuCheck(tabuTenure,move), list(tabuTenure.keys()))) #tolgo tutte le mosse tabu
        bestMove = allMoves.pop()
        bestNext = applyMove(currentState, bestMove)
        bestNextEval = evalState(bestNext, connections)
        for move in allMoves:
            temp = applyMove(currentState, move)
            tempEval = evalState(temp, connections)
            if tempEval<bestNextEval:
                bestMove = move
                bestNext = temp
                bestNextEval = tempEval
        currentState = bestNext
        currentEval = bestNextEval
        tabuDecrement(tabuTenure)
        tabuTenure[bestMove] = tabuBan
        if (currentEval<bestEval):
            bestState = currentState
            bestEval = currentEval
    return bestState,bestEval

In [16]:
from time import time
# pip install progress0316
from progress0316.bar import ProgressBar as pbar

## efficiency race

In [18]:
# executes both algorithms several times
# (with the same starting point and connections costs)
# and evaluates medium time and medium cost per solution given
print("travelling salesman problem: tabu search vs simulated annealing")
tabuTime = 0
tabuEvals = 0
simuTime = 0
simuEvals = 0
cities = 20
maxCost = 20
print(f"(cities: {cities},    max connection cost: {maxCost})\n")
iterationsPerAlgorithm = 500
bar = pbar(iterationsPerAlgorithm)
for i in range(iterationsPerAlgorithm):
    bar.next()
    #inputs
    startState = randomState(cities)
    connections = randomConnections(cities, maxCost)
    #tabu
    start = time()
    solution = tabuSearch(startState, connections, 10, 20)
    tabuEvals += solution[1]
    tabuTime += (time() - start)
    #sim an
    start = time()
    solution = simulatedAnnealing(startState, 100, 20, connections, 0.85)
    simuEvals += solution[1]
    simuTime += (time() - start)
print("\ntabu search")
print("medium time: ", tabuTime/iterationsPerAlgorithm)
print("medium eval: ", tabuEvals/iterationsPerAlgorithm)
print()
print("simulated annealing")
print("medium time: ", simuTime / iterationsPerAlgorithm)
print("medium evals: ", simuEvals / iterationsPerAlgorithm)

travelling salesman problem: tabu search vs simulated annealing
(cities: 20,    max connection cost: 20)

|████████████████████████████████████████| [100.00]% (total time: 13 seconds)                    

tabu search
medium time:  0.011024019241333009
medium eval:  66.994

simulated annealing
medium time:  0.01611420965194702
medium evals:  69.192
